# HexMagic

> Fill in a module description here

```python
#| default_exp core
```

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev import nbdev_export

In [ ]:
nbdev_export()

In [ ]:
#| export
#| hide
#import nbdev; nbdev.nbdev_export()
import sys
import math
from fastcore.basics import patch

#| export
## Introduction

The purpose of this library is to generate hex maps that are used in board games.

#| export
## Getting Started

In [ ]:
#| export
from HexMagic.plot.primitives import  MapCord , PrimitiveDemo
from HexMagic.plot.hex import Hex


from HexMagic.styles import StyleCSS,  SVGBuilder

In [ ]:
#| export
from HexMagic.primitives import MapPath, MapSize, MapRect, MapCord 
from HexMagic.primitives import HexGrid, HexPosition ,  HexRegion, GosperCurve, windy_edge , unique_windy_edge

import numpy as np

from HexMagic.terrain import Terrain
from HexMagic.voronoi import generate_plate_terrain


Terrain.fromSeeds = generate_plate_terrain
from HexMagic.climate import ClimatePreset, Climate, TerraDemo


In [ ]:
def demoTerr():

    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    
    sampleMap, plates =  Terrain.fromSeeds(myBounds,radius=15)

    sampleMap.colorMap()
    sgrid = sampleMap.hexGrid
    sgrid.builder.adjust("regions", sgrid.styleLayer(f=windy_edge(iterations=2, offset_factor=0.1)))
    #sampleMap.hexGrid.update()

    return sampleMap.hexGrid.builder.show()

In [ ]:
demoTerr()

In [ ]:
from HexMagic.terrainpatterns import TerrainPatterns
from HexMagic.climate import TerrainFactory

## Climate

In [ ]:

baMap = TerraDemo().bayAreaMap()

baMap.colorMap()
#baMap.hexGrid.update()
baMap.hexGrid.builder.layers = []
baMap.hexGrid.builder.adjust("regions", baMap.hexGrid.styleLayer(f=windy_edge(iterations=2, offset_factor=0.1)))
for layer in baMap.hexGrid.builder.layers:
    print(layer.name,layer)
baMap.hexGrid.builder.layers[-1].opacity = 0.1
baMap.climate.configure(baMap,debug=True)
baMap.add_climate_overlay()
baMap.hexGrid.builder.show()



In [ ]:
baMap.builder.layers = []
mountains = baMap.find_peaks(7,0,exclusion_radius=9)
for i , epicenter in enumerate(mountains):
    baMap.elevations += baMap.volcano(center=epicenter, adjusted=200+ ((i+1)*30), num_rings=6)
baMap.colorMap()
baMap.hexGrid.update()
baMap.builder.show()

## Hydrology

In [ ]:
from HexMagic.hydrology import DrainageBasins

In [ ]:
def hydrate(terrain):

    basin = DrainageBasins(terrain)

     
    terrain.hexGrid.builder.adjust("watersheds", basin.dotted_watershed_overlay(min_density=0.5))
    terrain.hexGrid.builder.adjust("borders",terrain.elevation_borders())

       # Add gradient flow lines
    gradient_overlay = basin.gradient_overlay(
        min_width=0.5,
        max_width=4.0,
        opacity=0.7
    )

    river_style = StyleCSS(
        "nile",
        fill = "none",
        stroke= '#23194629',
        stroke_width=3,
        opacity= 0.7
    )
    
    terrain.hexGrid.builder.add_style(river_style)
    river_svg = ""

    mainBasins = basin.get_major(6)
    
    for basin in mainBasins:
        small_river = basin.simplify(2)
        small_river.tributary.terrain = terrain
        river_svg += small_river.draw()

    terrain.hexGrid.builder.adjust("rivers", river_svg)



In [ ]:
baMap.climate.configure(baMap)
hydrate(baMap)
baMap.hexGrid.builder.show()

In [ ]:
??HexGrid.regions_by_value

In [ ]:
def showclimates(terrain,saturation=0.8):

    basin = DrainageBasins(terrain)

     
    terrain.hexGrid.builder.adjust("watersheds", basin.dotted_watershed_overlay(min_density=0.5))
    terrain.hexGrid.builder.adjust("borders",terrain.elevation_borders())

       # Add gradient flow lines
    gradient_overlay = basin.gradient_overlay(
        min_width=0.5,
        max_width=4.0,
        opacity=0.7
    )

    river_style = StyleCSS(
        "nile",
        fill = "none",
        stroke= '#23194629',
        stroke_width=3,
        opacity= 0.7
    )
    
    terrain.hexGrid.builder.add_style(river_style)
    river_svg = ""

    mainBasins = basin.get_major(6)
    
    for basin in mainBasins:
        small_river = basin.simplify(2)
        small_river.tributary.terrain = terrain
        river_svg += small_river.draw()

    sgrid = terrain.hexGrid
    sgrid.builder.layers = []




    climate_colors = {
        Climate.MARINE.value: "#1e88e5",
        Climate.FRESHWATER.value: "#42a5f5",
        Climate.TUNDRA.value: "#e3f2fd",
        Climate.DESERT.value: "#fdd835",
        Climate.GRASSLAND.value: "#9ccc65",
        Climate.FOREST.value: "#2e7d32",
        Climate.JUNGLE.value: "#1b5e20",
    }
    
    if 'climate' not in terrain.fields:
        terrain.compute_climate()
    
    climate_indices = terrain.fields['climate'].astype(int)
    styles = [StyleCSS(f"climate_{i}",fill = climate_colors[i]) for i in range(len(Climate))]
    #styles = [ for x in styles]
    for style in styles:
        style.properties["fill"] = style.desaturate(saturation)
        terrain.hexGrid.builder.add_style(style)

    
    #work
    sgrid = terrain.hexGrid
    print(climate_indices[:20])
    climateRegions = sgrid.regions_by_value(climate_indices)
    retLayer = ""
    
    borders = {}  # Shared cache across all regions
    
    overlay = ""
    for region in climateRegions:
    
        idx  =  region.hexes.pop()
        region.hexes.add(idx)
        
        styleI = int(climate_indices[idx])
        style = styles[styleI]
        print(idx,styleI,len(region.hexes))

        for path in region.trace_perimeter_cached(borders,
          style=style,
          f=unique_windy_edge(iterations=2)):
                overlay += path.svg()

    return overlay + river_svg
 


In [ ]:


overlay = showclimates(baMap,0.25)
baMap.hexGrid.builder.layers = []
baMap.hexGrid.builder.adjust("climate",overlay)
baMap.hexGrid.builder.show()



In [ ]:

from HexMagic.terraform import Terraform
from HexMagic.erosion import ErosionConfig,  ErosionSimulator
from HexMagic.styles import apply_looping_animation, LoopingLayerAnimation
import random

In [ ]:
??Terrain.volcano

In [ ]:
baMap = TerraDemo().bayAreaMap()
@patch
def longRun(self:TerraDemo, total_years=1000, snapshot_interval=100,debug = False):
    baMap = TerraDemo().bayAreaMap()
    baMap.climate.configure(baMap)
    siliconValley = Terraform(baMap)

    # Configure erosion
    config = ErosionConfig(
        years_per_iteration=20,  # Each iteration = 100 years
        iterations=1,              # Run one at a time
        debug=debug
    )
    org = baMap.clone()
    
    simulator = ErosionSimulator(siliconValley, config)
    
    # Store snapshots
    snapshots = []

    num_snapshots = total_years // snapshot_interval
    
    for i in range(num_snapshots):

        # Accumulate erosion for this interval
        snapshot_erosion = np.zeros(len(baMap.elevations))
        
        # Run erosion for this period
        simulator.simulate()
        mountains = random.sample(siliconValley.terrain.find_peaks(30,0,exclusion_radius=4),7)
        for i , epicenter in enumerate(mountains):
            siliconValley.add_event("volcano",
            adjustment = siliconValley.terrain.volcano(center=epicenter, adjusted=80, num_rings=3),
            name = f"starter_{epicenter}"
            )
        
        terrain = siliconValley.terrainFromEvents()


        terrain.climate.configure(terrain)
        terrain.colorMap()
        #overlay = siliconValley.terrain.hexGrid.styledHexes()
        overlay = terrain.hexGrid.styleLayer(f=windy_edge(iterations=2, offset_factor=0.1))
        overlay = showclimates(terrain,0.25)
        basin = DrainageBasins(terrain)
        river_svg = ""

        mainBasins = basin.get_major(8)
    
        for basin in mainBasins:
            small_river = basin.simplify(2)
            small_river.tributary.terrain = terrain
            overlay += small_river.draw()


        siliconValley.terrain = terrain
        
        #overlay += showclimates(siliconValley.terrain,0.25)
        snapshots.append(overlay)

        
    retTerrain = siliconValley.terrain.clone()
    retTerrain.builder.layers = []
    names = []
    for i , overlay in enumerate(snapshots):
        name = f"time_{i}"
        retTerrain.builder.adjust(f"time_{i}",overlay)
        names.append(name)

    anim = LoopingLayerAnimation(names, visible_count=2, step_duration=2, fade_duration=0.1, dim_opacity=0)
    apply_looping_animation( retTerrain.hexGrid.builder,anim)

    climate_colors = {
        Climate.MARINE.value: "#1e88e5",
        Climate.FRESHWATER.value: "#42a5f5",
        Climate.TUNDRA.value: "#e3f2fd",
        Climate.DESERT.value: "#fdd835",
        Climate.GRASSLAND.value: "#9ccc65",
        Climate.FOREST.value: "#2e7d32",
        Climate.JUNGLE.value: "#1b5e20",
    }
    

    
    saturation = 0.2
    styles = [StyleCSS(f"climate_{i}",fill = climate_colors[i]) for i in range(len(Climate))]
    #styles = [ for x in styles]
    for style in styles:
        style.properties["fill"] = style.desaturate(saturation)
        retTerrain.hexGrid.builder.add_style(style)

    return retTerrain



In [ ]:
allThings = TerraDemo().longRun(total_years=400, snapshot_interval=100)
allThings.hexGrid.builder.show()

In [ ]:
#overlay = showclimates(baMap,0.25)